In [1]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import cv2
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct
from random import shuffle as shuf
from imutils import paths
import os
import numpy as np
from PCA2D import PCA2D

In [3]:
# implement 2D DCT
def dct2(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

# implement 2D IDCT
def idct2(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho') 

def dct_compression(img):
    imf = np.float32(img)
    comp = dct2(imf)/255. # Scale to force low values to 0
    comp = np.uint8(idct2(comp)*255)
    return comp

###### def cv_disp_img(img, title="image"):
    while True:
        cv2.imshow(title, img)
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            # q key pressed so quit
            print("Quitting...")
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [5]:
imagePaths = list(paths.list_images("./Dataset/Train/"))
shuf(imagePaths)

In [8]:
len(imagePaths)

1990

In [9]:
# Generate Dataset Matrix & Normalize/DCT
width = height = 300
size = (width * height)

image_mat = np.zeros((len(imagePaths), height,width),dtype=np.uint8)

for ct, image in enumerate(imagePaths):
    img = cv2.imread(image, 0)
    scaled = cv2.resize(img, (width, height), interpolation=cv2.INTER_LINEAR)
    norm_img = np.zeros((width, height))
    norm_img = cv2.normalize(scaled, norm_img, 0, 255, cv2.NORM_MINMAX)
    dct_image = dct_compression(norm_img)
    image_mat[ct, :, :] = dct_image

In [10]:
# 2DPCA
def give_d(eig_vals,quality):
    s = np.sum(eig_vals)
    thresh = s * quality
    t = 0
    d = 0
    while t < thresh:
        t += eig_vals[d]
        d += 1
    return d
        
    
def reduce_dim_row(images,d=0,quality=0):
    A = np.mean(images,0)
    M = images.shape[0]
    
    height = images.shape[1]

    G = np.zeros((height, height))
    for i in range(M):
        G += (images[i]-A).T @ (images[i]-A)
    G /= M
    
    G_e_vals, G_e_vec = np.linalg.eig(G)
    
    if quality:
        d = give_d(G_e_vals, quality)
    
    X = G_e_vec[:,0:d]
    
    Y = images @ X
    
    return Y, X, G
    
    
def reduce_dim_col(images,q=0,quality=0):
    A = np.mean(images,0)
    M = images.shape[0]
    
    width = images.shape[2]

    H = np.zeros((width, width))
    for i in range(M):
        H += (images[i]-A) @ (images[i]-A).T
    H /= M
    
    H_e_vals, H_e_vec = np.linalg.eig(H)
    
    if quality:
        q = give_d(H_e_vals, quality)
    
    Z = H_e_vec[:,0:q]

    B = Z.T @ images
    
    return B, Z, H
    
    
    
def reduce_dim_square(images,q=0,d=0,quality=0):
    
    _,X,_ = reduce_dim_row(images,d=d,quality=quality)
    _,Z,_ = reduce_dim_col(images,q=q,quality=quality)
    
    C = Z.T @ images @ X
    
    return C, X, Z

In [11]:
Y, X, G = reduce_dim_row(image_mat,quality=.9)
B, Z, H = reduce_dim_col(image_mat,quality=.9)
C, X_2, Z_2 = reduce_dim_square(image_mat,quality=0.9)
# mean = np.mean(image_mat,0)
# C2,X2,Z2 = PCA2D(image_mat,mean,quality=.9).reduce_dim_col()

In [12]:
C.shape, X.shape, Z.shape

((1990, 7, 9), (300, 9), (300, 7))

In [13]:
np.save("./PP_Data/C.npy", C)
np.save("./PP_Data/X.npy", X)
np.save("./PP_Data/Z.npy", Z)

In [23]:
horiz = []
num = 0
for ct, image in enumerate(imagePaths):
    if "Connor" in image:
        if num <= 5:
            original = cv2.imread(image,0)
            original_scaled = cv2.resize(original, (height,width), interpolation=cv2.INTER_LINEAR)
            
            rowPCA = np.uint8(Y[ct] @ X.T)
            colPCA = np.uint8(Z @ B[ct])
            
            sqrPCA = np.uint8(Z @ C[ct] @ X.T)
            sqrPCA = cv2.resize(sqrPCA, (height,width), interpolation=cv2.INTER_LINEAR)
            
            featMat = cv2.resize(np.uint8(C[ct]), (height,width), interpolation=cv2.INTER_LINEAR)
        
            horiz += [cv2.hconcat((original_scaled,rowPCA,colPCA,sqrPCA,featMat))]
        num += 1

cv_disp_img(cv2.vconcat(tuple(horiz)))

Quitting...


In [24]:
def image_to_feature_vector(image):
    return np.uint8(image).flatten()

data = []
labels = []

for (i, imagePath) in enumerate(imagePaths):

    image = C[i]
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

    features = image_to_feature_vector(image)
    data.append(features)
    labels.append(label)

    if i > 0 and i % 10 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 10/1990
[INFO] processed 20/1990
[INFO] processed 30/1990
[INFO] processed 40/1990
[INFO] processed 50/1990
[INFO] processed 60/1990
[INFO] processed 70/1990
[INFO] processed 80/1990
[INFO] processed 90/1990
[INFO] processed 100/1990
[INFO] processed 110/1990
[INFO] processed 120/1990
[INFO] processed 130/1990
[INFO] processed 140/1990
[INFO] processed 150/1990
[INFO] processed 160/1990
[INFO] processed 170/1990
[INFO] processed 180/1990
[INFO] processed 190/1990
[INFO] processed 200/1990
[INFO] processed 210/1990
[INFO] processed 220/1990
[INFO] processed 230/1990
[INFO] processed 240/1990
[INFO] processed 250/1990
[INFO] processed 260/1990
[INFO] processed 270/1990
[INFO] processed 280/1990
[INFO] processed 290/1990
[INFO] processed 300/1990
[INFO] processed 310/1990
[INFO] processed 320/1990
[INFO] processed 330/1990
[INFO] processed 340/1990
[INFO] processed 350/1990
[INFO] processed 360/1990
[INFO] processed 370/1990
[INFO] processed 380/1990
[INFO] processed 390/

In [25]:
orig_labels = labels
le = LabelEncoder()
labels = le.fit_transform(labels)
num_labels=labels
print(orig_labels, labels)

['Connor', 'Maddie', 'Dad', 'Dad', 'Maddie', 'Mom', 'Dad', 'Dad', 'Connor', 'Connor', 'Maddie', 'Mom', 'Dad', 'Connor', 'Connor', 'Connor', 'Maddie', 'Mom', 'Mom', 'Connor', 'Mom', 'Mom', 'Maddie', 'Mom', 'Connor', 'Dad', 'Connor', 'Connor', 'Maddie', 'Dad', 'Connor', 'Maddie', 'Maddie', 'Connor', 'Connor', 'Maddie', 'Connor', 'Connor', 'Dad', 'Dad', 'Connor', 'Dad', 'Mom', 'Mom', 'Mom', 'Connor', 'Dad', 'Connor', 'Connor', 'Mom', 'Connor', 'Mom', 'Maddie', 'Mom', 'Maddie', 'Connor', 'Mom', 'Dad', 'Connor', 'Connor', 'Maddie', 'Connor', 'Connor', 'Mom', 'Dad', 'Connor', 'Connor', 'Connor', 'Dad', 'Mom', 'Mom', 'Connor', 'Connor', 'Connor', 'Maddie', 'Connor', 'Dad', 'Dad', 'Connor', 'Connor', 'Connor', 'Connor', 'Maddie', 'Dad', 'Mom', 'Mom', 'Connor', 'Connor', 'Dad', 'Mom', 'Mom', 'Connor', 'Dad', 'Connor', 'Connor', 'Mom', 'Connor', 'Mom', 'Connor', 'Dad', 'Connor', 'Connor', 'Connor', 'Connor', 'Mom', 'Connor', 'Maddie', 'Connor', 'Mom', 'Connor', 'Mom', 'Maddie', 'Mom', 'Connor', 

In [26]:
data = np.array(data) / 255.0
labels = np.uint8(np_utils.to_categorical(labels, 4))
print(labels[:5])

[[1 0 0 0]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]]


In [27]:
data.shape

(1990, 63)

In [28]:
np.save("./PP_Data/data.npy", data)
np.save("./PP_Data/labels.npy", labels)